In [38]:
import pandas as pd
import numpy as np
import random
import plotly as px
import plotly_express as go

import matplotlib.pyplot as plt
import seaborn as sns
from termcolor import colored
import warnings

color_pal = sns.color_palette("husl", 9)
warnings.filterwarnings('ignore')


import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots


In [39]:
import pandas as pd

# Load the CSV file without headers
df = pd.read_csv("AI modelling - DATA.csv", header=None)

# Assign new column names since they lack column names  
df.columns = ['Datetime', 'Temperature']
df

,Datetime,Temperature
0,2024-08-30 12:49:00,22.85
1,2024-08-30 12:50:00,22.85
2,2024-08-30 12:50:00,22.85
3,2024-08-30 12:51:00,22.81
4,2024-08-30 12:51:00,22.84
...,...,...
46980,2024-10-17 12:15:00,19.57
46981,2024-10-17 12:15:00,25.00
46982,2024-10-17 12:19:00,25.00
46983,2024-10-17 12:23:00,25.00


In [40]:
import pandas as pd

# Load the CSV file without headers
df = pd.read_csv("AI modelling - DATA.csv", header=None)

# Assign appropriate column names (assuming the first column is the datetime and the second is some value)
df.columns = ['Datetime', 'Temperature']

# Convert the 'Datetime' column to pandas datetime format
df['Datetime'] = pd.to_datetime(df['Datetime'], errors='coerce')

# Check for any rows where the conversion failed
print("Rows with invalid datetime values:")
print(df[df['Datetime'].isna()])

# Extract the day of the week if the conversion was successful
df['day_of_week'] = df['Datetime'].dt.day_name()

# Display the updated dataframe
print(df)


Rows with invalid datetime values:
Empty DataFrame
Columns: [Datetime, Temperature]
Index: []
                 Datetime  Temperature day_of_week
0     2024-08-30 12:49:00        22.85      Friday
1     2024-08-30 12:50:00        22.85      Friday
2     2024-08-30 12:50:00        22.85      Friday
3     2024-08-30 12:51:00        22.81      Friday
4     2024-08-30 12:51:00        22.84      Friday
...                   ...          ...         ...
46980 2024-10-17 12:15:00        19.57    Thursday
46981 2024-10-17 12:15:00        25.00    Thursday
46982 2024-10-17 12:19:00        25.00    Thursday
46983 2024-10-17 12:23:00        25.00    Thursday
46984 2024-10-17 12:25:00        19.62    Thursday

[46985 rows x 3 columns]


In [41]:
df.describe()

,Datetime,Temperature
count,46985,46985.000000
mean,2025-02-01 05:23:17.943279872,21.529063
min,2024-08-30 10:49:32,3.120000
25%,2024-09-11 08:41:00,18.450000
50%,2024-09-19 15:37:00,20.810000
75%,2024-10-01 21:04:00,22.060000
max,2106-02-07 09:28:15,655.350000
std,NaN,42.854934


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46985 entries, 0 to 46984
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Datetime     46985 non-null  datetime64[ns]
 1   Temperature  46985 non-null  float64       
 2   day_of_week  46985 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 1.1+ MB


# CREATE MODEL 

In [43]:
from sklearn.model_selection import train_test_split

# Define your feature matrix (X) and target variable (y)
X = df.drop(columns=['Datetime'])
y = df['Temperature']

#prepare taining set
X_train, X_test, y_train, y_test = train_test_split( X ,y , random_state=1, test_size=0.20)


In [44]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

# Define regression models
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree Regressor': DecisionTreeRegressor(),
    'Support Vector Regressor': SVR(),
    'K-Nearest Neighbors Regressor': KNeighborsRegressor(),
    'Random Forest Regressor': RandomForestRegressor(),
    'XGBoost Regressor': xgb.XGBRegressor()
}

# Train and evaluate models
for name, model in models.items():
    try:
        model.fit(X_train, y_train)
        score = model.score(X_test, y_test)
        print(f"{name}: {score}")
    except Exception as e:
        print(f"Error with {name}: {e}")

Error with Linear Regression: could not convert string to float: 'Friday'
Error with Decision Tree Regressor: could not convert string to float: 'Friday'
Error with Support Vector Regressor: could not convert string to float: 'Friday'
Error with K-Nearest Neighbors Regressor: could not convert string to float: 'Friday'
Error with Random Forest Regressor: could not convert string to float: 'Friday'
Error with XGBoost Regressor: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:day_of_week: object


In [45]:
from sklearn.linear_model import LinearRegression

# Create an SVR instance correctly
lf = LinearRegression()

# Fit the model
lf.fit(X_train, y_train)

# Print the score
print(lf.score(X_test, y_test))

# Predict on the test set
y_pred = lf.predict(X_test)

# Print predictions
print("Predictions of Average Temperatures Consumed:", y_pred)


ValueError: could not convert string to float: 'Friday'

# FEATURE ENGINEERING 

Creating New Features

In [8]:
# Convert the Datetime column to datetime format
df['Datetime'] = pd.to_datetime(df['Datetime'])

# Create new columns
df['Year'] = df['Datetime'].dt.year
df['Month'] = df['Datetime'].dt.month
df['Day'] = df['Datetime'].dt.day
df['Hour'] = df['Datetime'].dt.hour

# Define time of day categories
def categorize_time_of_day(hour):
    if 6 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 18:
        return 'Afternoon'
    elif 18 <= hour < 24:
        return 'Evening'
    else:
        return 'Night'

df['TimeOfDay'] = df['Hour'].apply(categorize_time_of_day)

# Add day of the week (0 = Monday, 6 = Sunday)
df['Weekday'] = df['Datetime'].dt.weekday

# Let's set the data types to numeric
df['Year'] = df['Year'].astype(int)
df['Weekday'] = df['Weekday'].astype(int)


df.tail()


,Datetime,Temperature,Year,Month,Day,Hour,TimeOfDay,Weekday
46979,2024-10-17 12:15:00,19.57,2024,10,17,12,Afternoon,3
46980,2024-10-17 12:15:00,25.00,2024,10,17,12,Afternoon,3
46981,2024-10-17 12:19:00,25.00,2024,10,17,12,Afternoon,3
46982,2024-10-17 12:23:00,25.00,2024,10,17,12,Afternoon,3
46983,2024-10-17 12:25:00,19.62,2024,10,17,12,Afternoon,3


# SEASONALITY 

In [28]:
import pandas as pd

# Load the CSV file without headers
df = pd.read_csv("AI modelling - DATA.csv", header=None)
# Assign appropriate column names (assuming the first column is the datetime and the second is some value)
df.columns = ['Datetime', 'Value']

# Convert the 'Datetime' column to pandas datetime format
df['Datetime'] = pd.to_datetime(df['Datetime'], errors='coerce')

# Check for any rows where the conversion failed
print("Rows with invalid datetime values:")
print(df[df['Datetime'].isna()])

# Extract the day of the week if the conversion was successful
df['day_of_week'] = df['Datetime'].dt.day_name()

# Display the updated dataframe
print(df)


Rows with invalid datetime values:
Empty DataFrame
Columns: [Datetime, Value]
Index: []
                 Datetime  Value day_of_week
0     2024-08-30 12:49:00  22.85      Friday
1     2024-08-30 12:50:00  22.85      Friday
2     2024-08-30 12:50:00  22.85      Friday
3     2024-08-30 12:51:00  22.81      Friday
4     2024-08-30 12:51:00  22.84      Friday
...                   ...    ...         ...
46980 2024-10-17 12:15:00  19.57    Thursday
46981 2024-10-17 12:15:00  25.00    Thursday
46982 2024-10-17 12:19:00  25.00    Thursday
46983 2024-10-17 12:23:00  25.00    Thursday
46984 2024-10-17 12:25:00  19.62    Thursday

[46985 rows x 3 columns]
